<a href="https://colab.research.google.com/github/hosh-farms/hosh_simulator/blob/main/hoshfarms_sowsimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import streamlit as st
import pandas as pd
import plotly.graph_objects as go

# --- Farm simulation function ---
def farm_simulator(
    total_sows=30,
    piglets_per_cycle=8,
    sow_mortality=0.033,
    piglet_mortality=0.03,
    sow_feed_price=32,
    sow_feed_intake=2.8,
    grower_feed_price=28,
    fcr=3.2,
    final_weight=105,
    sale_price=180,
    management_fee=50000,
    management_commission=0.05,
    supervisor_salary=25000,
    worker_salary=18000,
    n_workers=2,
    shed_cost=1_000_000,
    shed_life_years=10,
    sow_cost=1_050_000,
    sow_life_years=4,
    loan_amount=0,
    interest_rate=0.1,
    loan_tenure_years=5,
    moratorium_months=0,
    medicine_cost=10000,
    electricity_cost=5000,
    land_lease=10000,
    months=60
):
    current_sows = total_sows
    monthly_data = []

    # Depreciation
    shed_dep = shed_cost / (shed_life_years * 12)
    sow_dep_total = sow_cost / (sow_life_years * 12)

    # Loan EMI calculation
    total_months = loan_tenure_years * 12
    monthly_rate = interest_rate / 12
    emi = 0
    if loan_amount > 0 and total_months > 0:
        emi = loan_amount * monthly_rate * (1 + monthly_rate)**total_months / ((1 + monthly_rate)**total_months - 1)
    loan_balance = loan_amount

    # Sow rotation for batches (rough example: 6 sows one month, 5 next, repeat)
    rotation = [6,5] * (months // 2 + 1)
    batches = []

    for month in range(1, months + 1):
        sow_feed_cost = current_sows * sow_feed_intake * 30 * sow_feed_price
        staff_cost = supervisor_salary + n_workers * worker_salary
        mgmt_fixed = management_fee

        # Mate sows for this month
        sows_to_mate = min(rotation[month-1], round(current_sows))
        if sows_to_mate > 0:
            farrow_month = month + 4
            wean_month = farrow_month + 1
            grower_start_month = wean_month
            grower_end_month = grower_start_month + 6
            piglets = sows_to_mate * piglets_per_cycle * (1 - piglet_mortality)
            batches.append({
                'farrow_month': farrow_month,
                'wean_month': wean_month,
                'grower_start_month': grower_start_month,
                'grower_end_month': grower_end_month,
                'piglets': piglets
            })

        # Monthly counters
        piglets_with_sow = 0
        growers = 0
        sold_pigs = 0
        revenue = 0
        grower_feed_cost = 0

        for batch in batches:
            if batch['farrow_month'] <= month < batch['wean_month']:
                piglets_with_sow += batch['piglets']
            if batch['grower_start_month'] <= month < batch['grower_end_month']:
                growers += batch['piglets']
                grower_feed_cost += batch['piglets'] * fcr * final_weight / 6 * grower_feed_price
            if batch['grower_end_month'] == month:
                pigs_sold = batch['piglets'] * (1 - 0.03)
                revenue += pigs_sold * final_weight * sale_price
                sold_pigs += pigs_sold

        mgmt_comm_cost = revenue * management_commission
        other_fixed = medicine_cost + electricity_cost + land_lease
        total_operating_cost = sow_feed_cost + grower_feed_cost + staff_cost + mgmt_fixed + mgmt_comm_cost + other_fixed
        dep = shed_dep + sow_dep_total

        # Loan EMI
        if month <= moratorium_months:
            loan_payment = loan_balance * monthly_rate
        elif month <= total_months:
            interest = loan_balance * monthly_rate
            principal = emi - interest
            loan_balance -= principal
            loan_payment = emi
        else:
            loan_payment = 0

        monthly_data.append({
            'Month': month,
            'Piglets_Born_Alive': int(piglets_with_sow),
            'Growers': int(growers),
            'Sold_Pigs': int(sold_pigs),
            'Revenue': int(revenue),
            'Sow_Feed_Cost': int(sow_feed_cost),
            'Grower_Feed_Cost': int(grower_feed_cost),
            'Staff_Cost': int(staff_cost),
            'Mgmt_Fee': int(mgmt_fixed),
            'Mgmt_Comm': int(mgmt_comm_cost),
            'Other_Fixed_Costs': int(other_fixed),
            'Total_Operating_Cost': int(total_operating_cost),
            'Depreciation': int(dep),
            'Loan_EMI': int(loan_payment)
        })

        current_sows -= current_sows * sow_mortality / 12

    df_month = pd.DataFrame(monthly_data)
    df_month['Year'] = ((df_month['Month'] - 1) // 12) + 1

    cols_to_sum = ['Revenue','Sow_Feed_Cost','Grower_Feed_Cost','Staff_Cost','Mgmt_Fee','Mgmt_Comm',
                   'Other_Fixed_Costs','Total_Operating_Cost','Depreciation','Loan_EMI',
                   'Piglets_Born_Alive','Growers','Sold_Pigs']
    df_year = df_month.groupby('Year')[cols_to_sum].sum()
    df_year.index = [f'Year {i}' for i in df_year.index]
    df_year['Cash_Profit'] = df_year['Revenue'] - df_year['Total_Operating_Cost']
    df_year['Profit_After_Dep_Loan'] = df_year['Cash_Profit'] - df_year['Depreciation'] - df_year['Loan_EMI']

    return df_month, df_year

# --- Streamlit App ---
st.title("Breeding Sow Farm Simulator")

st.sidebar.header("Farm Parameters")

# Sliders
total_sows = st.sidebar.slider("Total Sows", 10, 100, 30)
piglets_per_cycle = st.sidebar.slider("Piglets per Cycle", 4, 20, 8)
sow_mortality = st.sidebar.slider("Sow Mortality %", 0.0, 0.1, 0.033)
piglet_mortality = st.sidebar.slider("Piglet Mortality %", 0.0, 0.1, 0.03)
sow_feed_price = st.sidebar.slider("Sow Feed Price (₹/kg)", 10, 100, 32)
sow_feed_intake = st.sidebar.slider("Sow Feed Intake (kg/day)", 1.0, 5.0, 2.8)
grower_feed_price = st.sidebar.slider("Grower Feed Price (₹/kg)", 10, 100, 28)
fcr = st.sidebar.slider("FCR", 3.1, 4.0, 3.2, 0.01)
final_weight = st.sidebar.slider("Final Market Weight (kg)", 50, 200, 105)
sale_price = st.sidebar.slider("Sale Price per kg (₹)", 100, 300, 180)
management_fee = st.sidebar.slider("Management Fee (₹/month)", 0, 200000, 50000)
management_commission = st.sidebar.slider("Management Commission %", 0.0, 0.2, 0.05)
supervisor_salary = st.sidebar.slider("Supervisor Salary (₹/month)", 10000, 100000, 25000)
worker_salary = st.sidebar.slider("Worker Salary (₹/month)", 10000, 50000, 18000)
n_workers = st.sidebar.slider("Number of Workers", 0, 10, 2)
shed_cost = st.sidebar.slider("Shed Construction Cost (₹)", 100000, 5000000, 1_000_000)
shed_life_years = st.sidebar.slider("Shed Life (Years)", 1, 20, 10)
sow_cost = st.sidebar.slider("Sow Purchase Cost (₹)", 500000, 5_000_000, 1_050_000)
sow_life_years = st.sidebar.slider("Sow Life (Years)", 1, 10, 4)
loan_amount = st.sidebar.slider("Loan Amount (₹)", 0, 5_000_000, 0)
interest_rate = st.sidebar.slider("Loan Interest Rate", 0.0, 0.2, 0.1)
loan_tenure_years = st.sidebar.slider("Loan Tenure (Years)", 1, 10, 5)
moratorium_months = st.sidebar.slider("Loan Moratorium Months", 0, 24, 0)
medicine_cost = st.sidebar.slider("Monthly Medicine Cost (₹)", 0, 50000, 10000)
electricity_cost = st.sidebar.slider("Monthly Electricity Cost (₹)", 0, 50000, 5000)
land_lease = st.sidebar.slider("Monthly Land Lease (₹)", 0, 50000, 10000)
months = st.sidebar.slider("Months to Simulate", 12, 120, 60)

# Run simulation
df_month, df_year = farm_simulator(
    total_sows, piglets_per_cycle, sow_mortality, piglet_mortality, sow_feed_price,
    sow_feed_intake, grower_feed_price, fcr, final_weight, sale_price, management_fee,
    management_commission, supervisor_salary, worker_salary, n_workers,
    shed_cost, shed_life_years, sow_cost, sow_life_years, loan_amount, interest_rate,
    loan_tenure_years, moratorium_months, medicine_cost, electricity_cost, land_lease, months
)

st.subheader("Monthly Table")
st.dataframe(df_month)

st.subheader("Yearly Summary")
st.dataframe(df_year)

# --- Graphs ---
st.subheader("Monthly Piglets/Growers/Sold Pigs")
df_plot = df_month[['Month','Piglets_Born_Alive','Growers','Sold_Pigs']].set_index('Month')
st.bar_chart(df_plot)

st.subheader("Cumulative Sold Pigs")
df_month['Cumulative_Sold'] = df_month['Sold_Pigs'].cumsum()
st.line_chart(df_month[['Cumulative_Sold']])

2025-08-28 23:03:05.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.443 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.453 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.454 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.457 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-08-28 23:03:05.459 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator()